In [1]:
import numpy as np
from scipy.stats import multivariate_normal

In [2]:
def F(theta):
    if theta.ndim == 1:
        theta = np.expand_dims(theta, 0)
    return -np.sum((theta - 0.5) ** 2, axis=tuple(range(theta.ndim)[1:]))

Updated Hessian Method

In [26]:
delta_g_lst = []
diag_lst = []
def ES_hessian(alpha, sigma, theta_0, num_samples, time_steps, p, H_lambda):
    d = theta_0.shape[0]
    theta_t = theta_0
    n = num_samples
    H = None
    for t in range(time_steps):
        #**** sample epsilons ****#
        epsilons = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d), size = n) # n by d
        #**** compute Hessian every p steps ****#
        if t % p == 0:
            Fs = F(theta_t + sigma*epsilons)
            eps = np.expand_dims(epsilons, -1)
            H_samples = ((eps*np.transpose(eps, (0, 2, 1)) - np.identity(d))* Fs.reshape(-1, 1, 1))/(sigma**2)
            H = H_samples.mean(axis=0)
        #**** update theta: compute g ****#
        u, s, vh = np.linalg.svd(H)
        H_nh = u @ np.diag(s**-0.5) @ vh
        H_nh_3d = np.ones((n, d, d)) * H_nh
        Fs = F(theta_t +  sigma* np.transpose((H_nh @ np.transpose(epsilons) )) ) - F(theta_t)
        eps = np.expand_dims(epsilons, -1)
        g_samples =  H_nh_3d @ eps * Fs.reshape(-1, 1, 1)/sigma
        g = g_samples.mean(axis=0).ravel()
        
        #**** update theta: the rest ****#
        new_theta = theta_t + alpha * g
        theta_t = new_theta
        
    return theta_t, F(theta_t), H

In [27]:
res = ES_hessian(alpha=0.5, sigma=1, theta_0=np.array([1.0,1.0]), num_samples = 1000, time_steps = 1000, p = 10, H_lambda = 0)
print(res)

(array([0.5275119 , 0.52577876]), array([-0.00142145]), array([[-2.00192311,  0.14673858],
       [ 0.14673858, -1.70061962]]))
